<h3>1. Import Libraries

In [152]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
!pip install folium
import folium
from geopy.geocoders import Nominatim  # module to convert an address into latitude and longitude values
!pip install lxml
import requests
import lxml.html as lh
from sklearn.cluster import KMeans
print("Libraries imported.")

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


<h3> 2. Scrapping Data from webpage

In [153]:
url='https://en.wikipedia.org/wiki/List_of_communities_in_Dubai'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
r = requests.get(url)

#Create empty list
col=[]
i=0
#For each row, store each first element (header) in the empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

size_of_col = 6
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 7, the //tr data is not from our table 
    if len(T)!=size_of_col:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

1:"Community Number
"
2:"Community (English)"
3:"Community (Arabic)"
4:"Area(km2)"
5:"Population(2000)"
6:"Population density(/km2)
"


,Community Number\n,Community (English),Community (Arabic),Area(km2),Population(2000),Population density(/km2)\n
0,126\n,Abu Hail,أبو هيل,1.27 km2,"21,414","16,861.4/km2\n"
1,711\n,Al Awir First,العوير الأولى,,,\n
2,721\n,Al Awir Second,العوير الثانية,,,\n
3,283\n,Aleyas,العياص,162.4 km2,"1,706",162.4/km2\n
4,333\n,Al Bada'a,البدع,0.82 km2,"18,816",22946/km2\n
...,...,...,...,...,...,...
147,913\n,Umm Nahad Third,,,,\n
148,914\n,Umm Nahad Fourth,,,,\n
149,971\n,Saih Al-Dahal,,,,\n
150,951\n,Saih Al Salam,,,,\n


<h3>3. Finding the latitude and longitude of each Community

In [154]:
df = df.drop(['Community (Arabic)','Area(km2)','Population(2000)','Population density(/km2)\n'], axis=1)
df

,Community Number\n,Community (English)
0,126\n,Abu Hail
1,711\n,Al Awir First
2,721\n,Al Awir Second
3,283\n,Aleyas
4,333\n,Al Bada'a
...,...,...
147,913\n,Umm Nahad Third
148,914\n,Umm Nahad Fourth
149,971\n,Saih Al-Dahal
150,951\n,Saih Al Salam


In [155]:
df.replace('\n','', regex=True, inplace=True)
df.columns= ['Community Number', 'Community (English)']
df

,Community Number,Community (English)
0,126,Abu Hail
1,711,Al Awir First
2,721,Al Awir Second
3,283,Aleyas
4,333,Al Bada'a
...,...,...
147,913,Umm Nahad Third
148,914,Umm Nahad Fourth
149,971,Saih Al-Dahal
150,951,Saih Al Salam


In [156]:
df["Latitude"] = ""
df["Longitude"] = ""
df["City"]= "Dubai"
df.shape

(152, 5)

In [157]:
df.head()

,Community Number,Community (English),Latitude,Longitude,City
0,126,Abu Hail,,,Dubai
1,711,Al Awir First,,,Dubai
2,721,Al Awir Second,,,Dubai
3,283,Aleyas,,,Dubai
4,333,Al Bada'a,,,Dubai


In [158]:
df1 = df.loc[0:25]
df2 = df.loc[26:50]
df3 = df.loc[51:75]
df4 = df.loc[76:100]
df5 = df.loc[101:125]
df6 = df.loc[126:145]

In [159]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row["Community (English)"] + ", Dubai"
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Abu Hail, Dubai are 25.2860287, 55.328865.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The geograpical coordinate of Al Awir First, Dubai are 25.185184200000002, 55.5651697615552.
Cannot do: Al Awir Second, Dubai, will drop index: 2
The geograpical coordinate of Aleyas, Dubai are 25.2117884, 55.536023378308464.
The geograpical coordinate of Al Bada'a, Dubai are 25.2246261, 55.2684831.
The geograpical coordinate of Al Baraha, Dubai are 25.2810618, 55.3194665.
The geograpical coordinate of Al Barsha First, Dubai are 25.1117235, 55.2064271.
Cannot do: Al Barsha Second, Dubai, will drop index: 7
Cannot do: Al Barsha Third, Dubai, will drop index: 8
The geograpical coordinate of Al Barsha South First, Dubai are 25.0747157, 55.2320479.
Cannot do: Al Barsha South Second, Dubai, will drop index: 10
Cannot do: Al Barsha South Third, Dubai, will drop index: 11
Cannot do: Al Barsha South Fourth, Dubai, will drop index: 12
Cannot do: Al Barsha South Fifth, Dubai, will drop index: 13
The geograpical coordinate of Al Buteen, Dubai are 25.26305655, 55.3205840389995.
The geograpical coo

In [160]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer2")
for index, row in df2.iterrows():
    address = row["Community (English)"] + ', Dubai '
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df2.loc[index, 'Latitude'] = latitude
        df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Al Hebiah Sixth, Dubai , will drop index: 26
Cannot do: Al Hebiah Third, Dubai , will drop index: 27
The geograpical coordinate of Al Hudaiba, Dubai  are 25.2428328, 55.279212.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The geograpical coordinate of Al Jaddaf, Dubai  are 25.2218696, 55.3359246.
The geograpical coordinate of Al Jafiliya, Dubai  are 25.2335944, 55.2921844.
The geograpical coordinate of Al Karama, Dubai  are 25.2482298, 55.3051575.
The geograpical coordinate of Al Khabisi, Dubai  are 25.2683331, 55.3368902.
The geograpical coordinate of Al Khawaneej First, Dubai  are 25.23997265, 55.50901189367384.
Cannot do: Al Khawaneej Second, Dubai , will drop index: 34
The geograpical coordinate of Al Kifaf, Dubai  are 25.2278867, 55.2748092.
The geograpical coordinate of Al Mamzar, Dubai  are 25.3146919, 55.341573715967506.
The geograpical coordinate of Al Manara, Dubai  are 25.1448185, 55.2144527.
The geograpical coordinate of Al Mankhool, Dubai  are 25.2495092, 55.2954769.
The geograpical coordinate of Al Merkad, Dubai  are 25.169213, 55.2924727.
The geograpical coordinate of Al Mina, Dubai  are 25.2496256, 55.276165.
The geograpical coordinate of Al Mizhar First, Dubai  are 25.2450842, 55.450315

In [161]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer3")
for index, row in df3.iterrows():
    address = row["Community (English)"] + ', Dubai '
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df3.loc[index, 'Latitude'] = latitude
        df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Al Quoz Industrial First, Dubai  are 25.1250785, 55.21444615.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Cannot do: Al Quoz Industrial Fourth, Dubai , will drop index: 52
Cannot do: Al Quoz Industrial Second, Dubai , will drop index: 53
Cannot do: Al Quoz Industrial Third, Dubai , will drop index: 54
Cannot do: Al Quoz Second, Dubai , will drop index: 55
Cannot do: Al Quoz Third, Dubai , will drop index: 56
Cannot do: Al Quoz Fourth, Dubai , will drop index: 57
The geograpical coordinate of Al Qusais First, Dubai  are 25.2627427, 55.3872735.
Cannot do: Al Qusais Industrial Fifth, Dubai , will drop index: 59
Cannot do: Al Qusais Industrial First, Dubai , will drop index: 60
Cannot do: Al Qusais Industrial Fourth, Dubai , will drop index: 61
Cannot do: Al Qusais Industrial Second, Dubai , will drop index: 62
The geograpical coordinate of Al Qusais Industrial Third, Dubai  are 25.3260017, 55.4331454.
Cannot do: Al Qusais Second, Dubai , will drop index: 64
The geograpical coordinate of Al Qusais Third, Dubai  are 25.3260017, 55.4331454.
The geograpical coordinate of Al Raffa, Dubai  are 25.2

In [162]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer4")
for index, row in df4.iterrows():
    address = row["Community (English)"] + ', Dubai '
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df4.loc[index, 'Latitude'] = latitude
        df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Al Shindagha, Dubai  are 25.2661597, 55.2888679.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The geograpical coordinate of Al Souq Al Kabeer, Dubai  are 25.2620458, 55.2936315.
The geograpical coordinate of Al Twar First, Dubai  are 25.262628, 55.3795909.
Cannot do: Al Twar Second, Dubai , will drop index: 79
Cannot do: Al Twar Third, Dubai , will drop index: 80
Cannot do: Al Warqa'a Fifth, Dubai , will drop index: 81
Cannot do: Al Warqa'a First, Dubai , will drop index: 82
Cannot do: Al Warqa'a Fourth, Dubai , will drop index: 83
Cannot do: Al Warqa'a Second, Dubai , will drop index: 84
The geograpical coordinate of Al Warqa'a Third, Dubai  are 25.190661, 55.4227553522109.
The geograpical coordinate of Al Wasl, Dubai  are 25.1959326, 55.2557371.
The geograpical coordinate of Al Waheda, Dubai  are 25.2915387, 55.3373622.
The geograpical coordinate of Ayal Nasir, Dubai  are 25.2743483, 55.3041029.
The geograpical coordinate of Business Bay, Dubai  are 25.1830272, 55.2729089.
The geograpical coordinate of Bu Kadra, Dubai  are 22.67, 55.92.
The geograpical coordinate of Downtown 

In [163]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer5")
for index, row in df5.iterrows():
    address = row["Community (English)"] + ', Dubai '
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df5.loc[index, 'Latitude'] = latitude
        df5.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Jebel Ali 1, Dubai  are 24.9775987, 55.0910742.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The geograpical coordinate of Jebel Ali 2, Dubai  are 24.9775987, 55.0910742.
The geograpical coordinate of Jebel Ali Industrial, Dubai  are 25.0025696, 55.12652395.
The geograpical coordinate of Jebel Ali Palm, Dubai  are 24.9848141, 55.0057904.
The geograpical coordinate of Jumeira First, Dubai  are 25.233175, 55.2773708.
The geograpical coordinate of Palm Jumeira, Dubai  are 25.1214019, 55.1290981.
The geograpical coordinate of Jumeira Second, Dubai  are 25.3063769, 55.3891306.
Cannot do: Jumeira Third, Dubai , will drop index: 108
The geograpical coordinate of Marsa Dubai, Dubai  are 25.0877537, 55.14617209862042.
The geograpical coordinate of Mirdif, Dubai  are 25.2211708, 55.4224204.
Cannot do: Muhaisanah Fourth, Dubai , will drop index: 111
Cannot do: Muhaisanah Second, Dubai , will drop index: 112
Cannot do: Muhaisanah Third, Dubai , will drop index: 113
The geograpical coordinate of Muhaisnah First, Dubai  are 25.2720137, 55.4364278.
Cannot do: Nad Al Hammar, Dubai , will drop

In [164]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer6")
for index, row in df6.iterrows():
    address = row["Community (English)"] + ', Dubai '
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df6.loc[index, 'Latitude'] = latitude
        df6.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Ras Al Khor Industrial Second, Dubai , will drop index: 126
Cannot do: Ras Al Khor Industrial Third, Dubai , will drop index: 127
The geograpical coordinate of Rigga Al Buteen, Dubai  are 25.2595231, 55.3195095.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The geograpical coordinate of Trade Centre 1, Dubai  are 32.55988935, -95.86352875.
The geograpical coordinate of Trade Centre 2, Dubai  are 48.1454921, 17.155080343656103.
The geograpical coordinate of Umm Al Sheif, Dubai  are 25.1319978, 55.204668.
The geograpical coordinate of Umm Hurair First, Dubai  are 25.2422117, 55.321784.
Cannot do: Umm Hurair Second, Dubai , will drop index: 133
The geograpical coordinate of Umm Ramool, Dubai  are 25.2310318, 55.3674888.
The geograpical coordinate of Umm Suqeim First, Dubai  are 25.1539087, 55.2075004.
Cannot do: Umm Suqeim Second, Dubai , will drop index: 136
Cannot do: Umm Suqeim Third, Dubai , will drop index: 137
Cannot do: Wadi Alamardi, Dubai , will drop index: 138
The geograpical coordinate of Warsan First, Dubai  are 25.1631537, 55.422077.
Cannot do: Warsan Second, Dubai , will drop index: 140
The geograpical coordinate of Yaraah, Dubai  are 14.424065, 44.011175.
The geograpical coordinate of Za'abeel First, Dubai  are 25.224255, 55.2

In [165]:
print(to_drop_unknown)

[2, 7, 8, 10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 34, 42, 48, 49, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 64, 72, 73, 74, 79, 80, 81, 82, 83, 84, 93, 97, 108, 111, 112, 113, 115, 116, 117, 123, 125, 126, 127, 133, 136, 137, 138, 140, 145]


In [166]:
dfa = df1.append(df2, ignore_index = True)
dfa = dfa.append(df3, ignore_index = True)
dfa = dfa.append(df4, ignore_index = True)
dfa = dfa.append(df5, ignore_index = True)
dfa = dfa.append(df6, ignore_index = True)
cleandf = dfa.drop(to_drop_unknown)

In [167]:
cleandf.shape

(90, 5)

In [168]:
tosearch_df = df.loc[to_drop_unknown]
tosearch_df.head()
tosearch_df.reset_index(drop=True)

,Community Number,Community (English),Latitude,Longitude,City
0,721,Al Awir Second,,,Dubai
1,376,Al Barsha Second,,,Dubai
2,375,Al Barsha Third,,,Dubai
3,672,Al Barsha South Second,,,Dubai
4,673,Al Barsha South Third,,,Dubai
5,681,Al Barsha South Fourth,,,Dubai
6,684,Al Barsha South Fifth,,,Dubai
7,674,Al Hebiah First,,,Dubai
8,682,Al Hebiah Fourth,,,Dubai
9,683,Al Hebiah Fifth,,,Dubai


In [169]:
tosearch_df.shape

(56, 5)

In [170]:
tosearch_df1 = tosearch_df.iloc[0:25]
tosearch_df2 = tosearch_df.iloc[26:50]
tosearch_df3 = tosearch_df.iloc[51:75]
tosearch_df4 = tosearch_df.iloc[76:82]

In [171]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown2 = []
geolocator = Nominatim(user_agent="ny_explorer8")
for index, row in tosearch_df1.iterrows():
    address = str(row['Community (English)']) + ', Dubai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        tosearch_df1.loc[index, 'Latitude'] = latitude
        tosearch_df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown2.append(index)

Cannot do: Al Awir Second, Dubai, will drop index: 2
Cannot do: Al Barsha Second, Dubai, will drop index: 7
Cannot do: Al Barsha Third, Dubai, will drop index: 8
Cannot do: Al Barsha South Second, Dubai, will drop index: 10
Cannot do: Al Barsha South Third, Dubai, will drop index: 11
Cannot do: Al Barsha South Fourth, Dubai, will drop index: 12
Cannot do: Al Barsha South Fifth, Dubai, will drop index: 13
Cannot do: Al Hebiah First, Dubai, will drop index: 22
Cannot do: Al Hebiah Fourth, Dubai, will drop index: 23
Cannot do: Al Hebiah Fifth, Dubai, will drop index: 24
Cannot do: Al Hebiah Second, Dubai, will drop index: 25
Cannot do: Al Hebiah Sixth, Dubai, will drop index: 26
Cannot do: Al Hebiah Third, Dubai, will drop index: 27
Cannot do: Al Khawaneej Second, Dubai, will drop index: 34
Cannot do: Al Mizhar Second, Dubai, will drop index: 42
Cannot do: Al Nahda Second, Dubai, will drop index: 48
Cannot do: Al Nasr, Dubai, Dubai, will drop index: 49
Cannot do: Al Quoz Industrial Fourth

In [172]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer9")
for index, row in tosearch_df2.iterrows():
    address = str(row['Community (English)']) + ', Dubai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        tosearch_df2.loc[index, 'Latitude'] = latitude
        tosearch_df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown2.append(index)

Cannot do: Al Qusais Industrial Second, Dubai, will drop index: 62
Cannot do: Al Qusais Second, Dubai, will drop index: 64
Cannot do: Al Safa Second, Dubai, will drop index: 72
Cannot do: Al Safouh First, Dubai, will drop index: 73
Cannot do: Al Safouh Second, Dubai, will drop index: 74
Cannot do: Al Twar Second, Dubai, will drop index: 79
Cannot do: Al Twar Third, Dubai, will drop index: 80
Cannot do: Al Warqa'a Fifth, Dubai, will drop index: 81
Cannot do: Al Warqa'a First, Dubai, will drop index: 82
Cannot do: Al Warqa'a Fourth, Dubai, will drop index: 83
Cannot do: Al Warqa'a Second, Dubai, will drop index: 84
Cannot do: Dubai Investment Park Second, Dubai, will drop index: 93
Cannot do: Enkhali, Dubai, will drop index: 97
Cannot do: Jumeira Third, Dubai, will drop index: 108
Cannot do: Muhaisanah Fourth, Dubai, will drop index: 111
Cannot do: Muhaisanah Second, Dubai, will drop index: 112
Cannot do: Muhaisanah Third, Dubai, will drop index: 113
Cannot do: Nad Al Hammar, Dubai, will

In [173]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer10")
for index, row in tosearch_df3.iterrows():
    address = str(row['Community (English)']) + ', Dubai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        tosearch_df3.loc[index, 'Latitude'] = latitude
        tosearch_df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown2.append(index)

Cannot do: Umm Suqeim Second, Dubai, will drop index: 136
Cannot do: Umm Suqeim Third, Dubai, will drop index: 137
Cannot do: Wadi Alamardi, Dubai, will drop index: 138
Cannot do: Warsan Second, Dubai, will drop index: 140
Cannot do: Umm Nahad First, Dubai, will drop index: 145


In [174]:
# Need to drop those Neighborhood that the geocode does not find
geolocator = Nominatim(user_agent="ny_explorer11")
for index, row in tosearch_df4.iterrows():
    address = str(row['Community (English)']) + ', Dubai'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        tosearch_df4.loc[index, 'Latitude'] = latitude
        tosearch_df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown2.append(index)

In [175]:
print(to_drop_unknown2)

[2, 7, 8, 10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 34, 42, 48, 49, 52, 53, 54, 55, 56, 57, 59, 60, 62, 64, 72, 73, 74, 79, 80, 81, 82, 83, 84, 93, 97, 108, 111, 112, 113, 115, 116, 117, 123, 125, 126, 127, 136, 137, 138, 140, 145]


In [176]:
dfb = tosearch_df1.append(tosearch_df2, ignore_index = True)
dfb = dfb.append(tosearch_df3, ignore_index = True)
dfb = dfb.append(tosearch_df4, ignore_index = True)
dfb

,Community Number,Community (English),Latitude,Longitude,City
0,721,Al Awir Second,,,Dubai
1,376,Al Barsha Second,,,Dubai
2,375,Al Barsha Third,,,Dubai
3,672,Al Barsha South Second,,,Dubai
4,673,Al Barsha South Third,,,Dubai
5,681,Al Barsha South Fourth,,,Dubai
6,684,Al Barsha South Fifth,,,Dubai
7,674,Al Hebiah First,,,Dubai
8,682,Al Hebiah Fourth,,,Dubai
9,683,Al Hebiah Fifth,,,Dubai


In [177]:
dfa['Latitude'].replace('', np.nan, inplace=True)
dfa.dropna(subset=['Latitude'], inplace=True)
dfa.replace('\n','', regex=True)
dfa

,Community Number,Community (English),Latitude,Longitude,City
0,126,Abu Hail,25.286029,55.3289,Dubai
1,711,Al Awir First,25.185184,55.5652,Dubai
3,283,Aleyas,25.211788,55.536,Dubai
4,333,Al Bada'a,25.224626,55.2685,Dubai
5,122,Al Baraha,25.281062,55.3195,Dubai
...,...,...,...,...,...
139,621,Warsan First,25.163154,55.4221,Dubai
141,861,Yaraah,14.424065,44.0112,Dubai
142,325,Za'abeel First,25.224255,55.2864,Dubai
143,337,Za'abeel Second,-6.177421,35.75,Dubai


In [178]:
dfa.to_csv('DubaiLatLongNew.csv')

<h3>4. Define Foursquare credentials and versions

In [182]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ZXX1NUPBMWS1DFJ5YS4W5OOZ4D5CQCGOGE3K21YNLAMJNXXG' # your Foursquare ID
CLIENT_SECRET = 'JALWJBAX1VS1Z4RPPIIHPE2F3GUTUHXE4JYEAJ530C3VTCD2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
clean_df_new = dfa.copy()

Your credentails:
CLIENT_ID: ZXX1NUPBMWS1DFJ5YS4W5OOZ4D5CQCGOGE3K21YNLAMJNXXG
CLIENT_SECRET:JALWJBAX1VS1Z4RPPIIHPE2F3GUTUHXE4JYEAJ530C3VTCD2


In [183]:
clean_df_new.columns=['Community Number','Community (English)','Latitude','Longitude', 'City']
clean_df_new

,Community Number,Community (English),Latitude,Longitude,City
0,126,Abu Hail,25.286029,55.3289,Dubai
1,711,Al Awir First,25.185184,55.5652,Dubai
3,283,Aleyas,25.211788,55.536,Dubai
4,333,Al Bada'a,25.224626,55.2685,Dubai
5,122,Al Baraha,25.281062,55.3195,Dubai
...,...,...,...,...,...
139,621,Warsan First,25.163154,55.4221,Dubai
141,861,Yaraah,14.424065,44.0112,Dubai
142,325,Za'abeel First,25.224255,55.2864,Dubai
143,337,Za'abeel Second,-6.177421,35.75,Dubai


In [184]:
radius = 600
LIMIT = 200
venues = []

for lat, long, pin, post, city in zip(clean_df_new['Latitude'], clean_df_new['Longitude'], clean_df_new['Community Number'], clean_df_new['Community (English)'], clean_df_new['City']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,            pin,            lat,             long,          city,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        venues_df = pd.DataFrame(venues)
        venues_df.head()

In [185]:
# define the column names
venues_df.columns = ['Community (English)', 'Community Number', 'Latitude', 'Longitude', 'City','VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1538, 9)


,Community (English),Community Number,Latitude,Longitude,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,126,25.286029,55.328865,Dubai,Al Douri Mart Supermarket & Roastery,25.285869,55.328174,Supermarket
1,Abu Hail,126,25.286029,55.328865,Dubai,Baithak Restaurant,25.288937,55.327372,Asian Restaurant
2,Abu Hail,126,25.286029,55.328865,Dubai,Lively,25.285194,55.325276,Track
3,Abu Hail,126,25.286029,55.328865,Dubai,Jannati Health Club and Spa,25.285408,55.325168,Spa
4,Abu Hail,126,25.286029,55.328865,Dubai,Hamriya Park,25.285710,55.333000,Park


In [186]:
venues_df.to_csv('dubaiexplorenew.csv')

In [187]:
venues_df.groupby(['Community (English)', 'Community Number', 'City']).head()

,Community (English),Community Number,Latitude,Longitude,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,126,25.286029,55.328865,Dubai,Al Douri Mart Supermarket & Roastery,25.285869,55.328174,Supermarket
1,Abu Hail,126,25.286029,55.328865,Dubai,Baithak Restaurant,25.288937,55.327372,Asian Restaurant
2,Abu Hail,126,25.286029,55.328865,Dubai,Lively,25.285194,55.325276,Track
3,Abu Hail,126,25.286029,55.328865,Dubai,Jannati Health Club and Spa,25.285408,55.325168,Spa
4,Abu Hail,126,25.286029,55.328865,Dubai,Hamriya Park,25.285710,55.333000,Park
...,...,...,...,...,...,...,...,...,...
1531,Za'abeel Second,337,-6.177421,35.749954,Dubai,Younique Pub Dodoma,-6.180089,35.749450,Pub
1532,Za'abeel Second,337,-6.177421,35.749954,Dubai,Nyerere Square,-6.179483,35.752127,Park
1533,Za'abeel Second,337,-6.177421,35.749954,Dubai,Masterpub,-6.178891,35.753326,Pub
1534,Za'abeel Second,337,-6.177421,35.749954,Dubai,Double M,-6.178769,35.753961,Electronics Store


In [188]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 209 uniques categories.


In [189]:
venues_df['VenueCategory'].unique()[:20]

array(['Supermarket', 'Asian Restaurant', 'Track', 'Spa', 'Park', 'Beach',
       'Post Office', 'Coffee Shop', 'Seafood Restaurant',
       'Convenience Store', 'Fast Food Restaurant', 'Pool',
       'Indian Restaurant', 'Café', 'Tailor Shop',
       'Middle Eastern Restaurant', 'Restaurant', 'Hotel',
       'Turkish Restaurant', 'Lounge'], dtype=object)

<h2>5. Analyze Each Community For Venue Category

In [190]:
# one hot encoding by get dummies method
dubai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add Community,number and City column back to dataframe
dubai_onehot['Community Number'] = venues_df['Community Number'] 
dubai_onehot['Community (English)'] = venues_df['Community (English)'] 
dubai_onehot['City'] = venues_df['City'] 

# move community,number,city column to the first column
fixed_columns = list(dubai_onehot.columns[-3:]) + list(dubai_onehot.columns[:-3])
dubai_onehot = dubai_onehot[fixed_columns]

print(dubai_onehot.shape)
dubai_onehot.head()

(1538, 212)


,Community Number,Community (English),City,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Volleyball Court,Watch Shop,Water Park,Wings Joint,Women's Store,Yemeni Restaurant,Yoga Studio
0,126,Abu Hail,Dubai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,126,Abu Hail,Dubai,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,126,Abu Hail,Dubai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,126,Abu Hail,Dubai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,126,Abu Hail,Dubai,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
dubai_grouped = dubai_onehot.groupby(["Community Number", "Community (English)", "City"]).mean().reset_index()
print(dubai_grouped.shape)
dubai_grouped.head()

(80, 212)


,Community Number,Community (English),City,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Volleyball Court,Watch Shop,Water Park,Wings Joint,Women's Store,Yemeni Restaurant,Yoga Studio
0,,Downtown Dubai,Dubai,0.000000,0.0,0.0,0.0,0.0,0.0,0.026316,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026316
1,111,Al Corniche,Dubai,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,112,Al Ras,Dubai,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.043478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,113,Al Dhagaya,Dubai,0.117647,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.058824,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,114,Al Buteen,Dubai,0.000000,0.0,0.0,0.0,0.0,0.0,0.017857,...,0.000000,0.017857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


<h2>6. List and display the top 5 existing facilities for each Community Number

In [192]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ["Community Number", "Community (English)", "City"]
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Community Number'] = dubai_grouped['Community Number']
neighborhoods_venues_sorted['Community (English)'] = dubai_grouped['Community (English)']
neighborhoods_venues_sorted['City'] = dubai_grouped['City']

for ind in np.arange(dubai_grouped.shape[0]):
    row_categories = dubai_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(80, 8)


,Community Number,Community (English),City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,,Downtown Dubai,Dubai,Hotel,Coffee Shop,Café,Middle Eastern Restaurant,French Restaurant
1,111,Al Corniche,Dubai,French Restaurant,Bakery,Pizza Place,Flower Shop,Food Court
2,112,Al Ras,Dubai,History Museum,Jewelry Store,Middle Eastern Restaurant,Farmers Market,Café
3,113,Al Dhagaya,Dubai,Afghan Restaurant,Jewelry Store,Hotel,Men's Store,Fast Food Restaurant
4,114,Al Buteen,Dubai,Hotel,Restaurant,Middle Eastern Restaurant,Coffee Shop,Ice Cream Shop
...,...,...,...,...,...,...,...,...
75,600,Arabian Ranches,Dubai,Restaurant,Sports Bar,Golf Course,Pool,Yoga Studio
76,616,Nadd Al Shiba Third,Dubai,Café,Coffee Shop,Middle Eastern Restaurant,Turkish Restaurant,Gym
77,621,Warsan First,Dubai,Chinese Restaurant,Fast Food Restaurant,Indian Restaurant,Moroccan Restaurant,Gym
78,711,Al Awir First,Dubai,Fast Food Restaurant,Seafood Restaurant,Coffee Shop,Convenience Store,Yoga Studio


In [193]:
# Set manually to get proper fit in the map
address = 'Dubai'
latitude =  25.276987
longitude =  25.276987
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Dubai are 25.276987, 25.276987.


In [194]:
dubai_merged = dfa.copy()
dubai_merged = dubai_merged.join(neighborhoods_venues_sorted[["Community Number", "1st Most Common Venue"]].set_index("Community Number"), on="Community Number")
print(dubai_merged.shape)
dubai_merged

(90, 6)


,Community Number,Community (English),Latitude,Longitude,City,1st Most Common Venue
0,126,Abu Hail,25.286029,55.3289,Dubai,Park
1,711,Al Awir First,25.185184,55.5652,Dubai,Fast Food Restaurant
3,283,Aleyas,25.211788,55.536,Dubai,NaN
4,333,Al Bada'a,25.224626,55.2685,Dubai,Park
5,122,Al Baraha,25.281062,55.3195,Dubai,Turkish Restaurant
...,...,...,...,...,...,...
139,621,Warsan First,25.163154,55.4221,Dubai,Chinese Restaurant
141,861,Yaraah,14.424065,44.0112,Dubai,NaN
142,325,Za'abeel First,25.224255,55.2864,Dubai,Hotel
143,337,Za'abeel Second,-6.177421,35.75,Dubai,Pub


In [195]:
dubai_merged.to_csv('1st Most Common Venue')

<h2>7. Exploratory Visualization

In [196]:
import folium
my_map = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label1,common in zip(dubai_merged['Latitude'], dubai_merged['Longitude'], dubai_merged['Community (English)'],dubai_merged['1st Most Common Venue'] ):
    labelnew =  'Community: {} , Top Existing Infrastructure  : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(my_map)  
my_map

In [197]:
my_map.save('Map1.png')

<h2>8. Feature Engineering for Business Problem

In [198]:
venues_df['VenueCategory'].unique()

array(['Supermarket', 'Asian Restaurant', 'Track', 'Spa', 'Park', 'Beach',
       'Post Office', 'Coffee Shop', 'Seafood Restaurant',
       'Convenience Store', 'Fast Food Restaurant', 'Pool',
       'Indian Restaurant', 'Café', 'Tailor Shop',
       'Middle Eastern Restaurant', 'Restaurant', 'Hotel',
       'Turkish Restaurant', 'Lounge', 'IT Services', 'Ice Cream Shop',
       'Egyptian Restaurant', 'Bakery', 'Burrito Place',
       'Afghan Restaurant', 'Juice Bar', 'Pakistani Restaurant',
       'Shawarma Place', 'Bed & Breakfast', 'Fried Chicken Joint',
       'Watch Shop', 'Sports Bar', 'Pizza Place', 'Nightclub',
       'Hookah Bar', 'Athletics & Sports', 'Dumpling Restaurant',
       'Chinese Restaurant', 'Massage Studio', 'Szechuan Restaurant',
       'Thai Restaurant', 'Bar', 'Salon / Barbershop', 'Gym',
       'Currency Exchange', 'Food Court', 'Korean Restaurant',
       'Burger Joint', 'Hotel Bar', 'Department Store',
       'Syrian Restaurant', 'Iraqi Restaurant', 'Breakf

In [199]:
# Quality Infrastructure 
search_query= ['Restaurant', 'Hotel', 'Farmers Market', 'Shopping Mall', 'Gym / Fitness Center', 'Pharmacy',
                         'Electronics Store', 'Indie Movie Theater', 'Light Rail Station','Metro Station', 'Train','Train Station', 'Garden',
                          'Theater','ATM', 'Office', 'Bus Station', 'Bank', 'Market' , 'Business Service', 'Monument / Landmark' ,
                          'Resort', 'Hospital', 'Police Station', 'School', 'College', 'Café' , 'Park', 'Playground',
                'Convention Center', 'College Auditorium', 'Government Building', 'Airport Terminal',
                         ]
print(len(search_query))

33


In [200]:
quality_dataframe = []
quality_dataframe= venues_df.loc[venues_df['VenueCategory'].isin(search_query)]
quality_dataframe.shape

(396, 9)

In [201]:
quality_dataframe

,Community (English),Community Number,Latitude,Longitude,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
4,Abu Hail,126,25.286029,55.328865,Dubai,Hamriya Park,25.285710,55.333000,Park
5,Abu Hail,126,25.286029,55.328865,Dubai,Pond Park - Al Qusais,25.288060,55.332606,Park
14,Al Bada'a,333,25.224626,55.268483,Dubai,Al Bada'a Park,25.224104,55.268313,Park
15,Al Bada'a,333,25.224626,55.268483,Dubai,cafe aroma,25.226160,55.270326,Café
17,Al Bada'a,333,25.224626,55.268483,Dubai,Al Wasl Park (حديقة الوصل),25.222958,55.265674,Park
...,...,...,...,...,...,...,...,...,...
1525,Za'abeel First,325,25.224255,55.286425,Dubai,Novotel World Trade Centre Dubai,25.221415,55.288294,Hotel
1528,Za'abeel First,325,25.224255,55.286425,Dubai,Entre-Nous,25.221485,55.288592,Restaurant
1532,Za'abeel Second,337,-6.177421,35.749954,Dubai,Nyerere Square,-6.179483,35.752127,Park
1534,Za'abeel Second,337,-6.177421,35.749954,Dubai,Double M,-6.178769,35.753961,Electronics Store


In [202]:
# one hot encoding
qualitydubai_onehot = pd.get_dummies(quality_dataframe[['VenueCategory']], prefix="", prefix_sep="")
# add Community number, community,city column back to dataframe
qualitydubai_onehot['Community Number'] = quality_dataframe['Community Number'] 
qualitydubai_onehot['Community (English)'] = quality_dataframe['Community (English)'] 
qualitydubai_onehot['City'] = quality_dataframe['City'] 

# move these 3 to the first column
fixed_columns = list(qualitydubai_onehot.columns[-3:]) + list(qualitydubai_onehot.columns[:-3])
qualitydubai_onehot = qualitydubai_onehot[fixed_columns]

print(qualitydubai_onehot.shape)
qualitydubai_onehot.head()
print(qualitydubai_onehot.columns.values)

(396, 22)
['Community Number' 'Community (English)' 'City' 'Bank' 'Bus Station'
 'Business Service' 'Café' 'Electronics Store' 'Farmers Market'
 'Gym / Fitness Center' 'Hotel' 'Indie Movie Theater' 'Light Rail Station'
 'Market' 'Metro Station' 'Office' 'Park' 'Pharmacy' 'Playground' 'Resort'
 'Restaurant' 'Shopping Mall']


In [203]:
qualitydubai_grouped = qualitydubai_onehot.groupby(["Community (English)", "Community Number", "City"]).sum().reset_index()
print(qualitydubai_grouped.shape)
qualitydubai_grouped.head()

(67, 22)


,Community (English),Community Number,City,Bank,Bus Station,Business Service,Café,Electronics Store,Farmers Market,Gym / Fitness Center,...,Light Rail Station,Market,Metro Station,Office,Park,Pharmacy,Playground,Resort,Restaurant,Shopping Mall
0,Abu Hail,126,Dubai,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
1,Al Bada'a,333,Dubai,0,0,0,1,0,0,0,...,0,0,0,0,2,0,0,0,1,0
2,Al Baraha,122,Dubai,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Al Barsha First,373,Dubai,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Al Buteen,114,Dubai,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,4,1


In [204]:
qualitydubai_grouped['Total infrastructure'] =  qualitydubai_grouped[qualitydubai_grouped.drop(['Community (English)','Community Number','City'], axis=1).columns.values].sum(axis=1)

In [205]:
qualitydubai_grouped.shape

(67, 23)

<h2>What are the best locations in dubai as per infrastructure?

In [206]:
qualitydubai_grouped[qualitydubai_grouped['Total infrastructure'] == qualitydubai_grouped['Total infrastructure'].max()].transpose()

,59
Community (English),Rigga Al Buteen
Community Number,125
City,Dubai
Bank,1
Bus Station,0
Business Service,0
Café,3
Electronics Store,0
Farmers Market,0
Gym / Fitness Center,0


<h2>Which all areas lack the infrastructure facilities?

In [243]:
badquality = qualitydubai_grouped[qualitydubai_grouped['Total infrastructure'] == qualitydubai_grouped['Total infrastructure'].min()].transpose()
badquality

,0,5,8,10,11,15,18,19,20,21,...,27,28,30,33,35,36,43,44,47,48
Community (English),Abu Hail,Al Garhoud,Al Jafiliya,Al Mamzar,Al Mankhool,Al Mushrif,Al Raffa,Al Ras,Al Rashidiya,Al Rigga,...,Al Warqa'a Third,Al Wasl,Business Bay,Hor Al Anz,Jumeira First,Jumeira Second,Trade Centre 2,Umm Al Sheif,Warsan First,Za'abeel First
Acupuncturist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Breakfast Spot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Burger Joint,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comfort Food Restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dentist's Office,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Doctor's Office,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Emergency Room,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Fast Food Restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Food Court,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>9. Potential area for the development of infrastructure of different kinds</h2>
<h4>Looking for area in development of infrastructure in Community.</h4>
    
<h5>1. My choice of infrastructur for which Community has highest potential</h5>

In [208]:
yourchoiceinfra = 'Restaurant' # my choice of infrastructue from VenueCategory
badqualitychoice = qualitydubai_grouped[qualitydubai_grouped[yourchoiceinfra] == qualitydubai_grouped[yourchoiceinfra].min()]
badqualitychoice['Community (English)']      

0                 Abu Hail
2                Al Baraha
5               Al Dhagaya
8               Al Hudaiba
9                Al Jaddaf
12              Al Khabisi
13                Al Kifaf
14               Al Mamzar
15               Al Manara
17               Al Merkad
18                 Al Mina
19         Al Mizhar First
21                Al Murar
22              Al Mushrif
29                Al Raffa
30                  Al Ras
31            Al Rashidiya
33               Al Sabkha
34           Al Safa First
35                Al Satwa
37       Al Souq Al Kabeer
39        Al Warqa'a Third
43          Downtown Dubai
45     Emirates Hill First
46    Emirates Hill Second
49             Jebel Ali 1
50             Jebel Ali 2
51           Jumeira First
54         Muhaisnah First
55              Nad Shamma
57                    Naif
58              Port Saeed
62            Umm Al Sheif
63        Umm Hurair First
66         Za'abeel Second
Name: Community (English), dtype: object

<h5>2. my choice of area for which one infrastructure has highest potential

In [209]:
yourchoicearea = 'Al Karama'   # Change with the name of Community where I want to see potential
infraqualitychoice = qualitydubai_grouped[qualitydubai_grouped['Community (English)'] == yourchoicearea].transpose()
infraqualitychoice = infraqualitychoice.reset_index()

In [210]:
print("These are infrastructures with highest potential in" , yourchoicearea, "area : " )
for i in range(len(infraqualitychoice)) : 
    if (infraqualitychoice.iloc[i, 1] == 0):
        print(infraqualitychoice.iloc[i, 0])

These are infrastructures with highest potential in Al Karama area : 
Bank
Bus Station
Business Service
Electronics Store
Farmers Market
Indie Movie Theater
Light Rail Station
Market
Metro Station
Office
Park
Pharmacy
Playground
Resort
Shopping Mall


<h5>10.Best place to stay within a city for vital infrastructure facilities 

In [211]:
search_query2= ['Hospital','Food', 'Hotel', 'Shopping Mall', 'Pharmacy', 
                         'Metro Station', 'Train Station', 'ATM', 'Office', 'Bus Station', 'Bank', 'Market' ,
                          'Police Station', 'School', 'College & University', 'Park'
 ]
categoryId = ['4bf58dd8d48988d104941735','4d4b7105d754a06374d81259', '4bf58dd8d48988d1fa931735', '4bf58dd8d48988d1fd941735', '4bf58dd8d48988d10f951735', 
             '4bf58dd8d48988d1fd931735', '4bf58dd8d48988d129951735', '52f2ab2ebcbc57f1066b8b56', '4bf58dd8d48988d124941735','4bf58dd8d48988d1fe931735',
             '4bf58dd8d48988d10a951735', '50be8ee891d4fa8dcc7199a7','4bf58dd8d48988d12e941735', '4bf58dd8d48988d13b941735', '4d4b7105d754a06372d81259',
             '4bf58dd8d48988d163941735']

In [212]:
from pandas.io.json import json_normalize
radius = 500
VERSION = 20180604
# Quality Infrastructure 
search_query2 = 'Park'
categoryId = '4bf58dd8d48988d163941735'
LIMIT = 1

In [213]:
def getNearbyVenues(names, lat1, long1, radius):
    venues_list=[]
    for name, lat, lng in zip(names, lat1, long1):

        # create the API request URL
        url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&locale={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query2, radius, LIMIT,  'en', categoryId)
        # make the GET request
        results = requests.get(url1).json()["response"]["venues"]
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    return(nearby_venues)

In [214]:
names=dfa['Community (English)']
latitudes=dfa['Latitude']
longitudes=dfa['Longitude']
all_venues = getNearbyVenues(names,latitudes, longitudes, radius )

In [215]:
# convert the venues list into a new DataFrame
# define the column names
all_venues.columns = ['Community (English)','Latitude', 'Longitude','VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(all_venues.shape)
all_venues.head()

(28, 7)


,Community (English),Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.286029,55.328865,Hamriya Park,25.285710,55.333000,Park
1,Al Bada'a,25.224626,55.268483,Al Wasl Park (حديقة الوصل),25.222958,55.265674,Park
2,Al Baraha,25.281062,55.319466,Watchtower Park,25.276902,55.316955,Park
3,Al Buteen,25.263057,55.320584,Union Metro Park,25.263787,55.318791,Park
4,Al Garhoud,25.240737,55.352039,Garhoud Park,25.240850,55.349180,Park


In [216]:
search_query2 = 'Hospital'
categoryId = '4bf58dd8d48988d104941735'
LIMIT = 1

In [217]:
def getNearbyVenues(names, lat1, long1, radius):
    venues_list=[]
    for name, lat, lng in zip(names, lat1, long1):

        # create the API request URL
        url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&locale={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query2, radius, LIMIT,  'en', categoryId)
        # make the GET request
        results = requests.get(url2).json()["response"]["venues"]
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    return(nearby_venues)

In [218]:
names=dfa['Community (English)']
latitudes=dfa['Latitude']
longitudes=dfa['Longitude']
all_venues1 = getNearbyVenues(names,latitudes, longitudes, radius )

In [219]:
# convert the venues list into a new DataFrame
# define the column names
all_venues1.columns = ['Community (English)','Latitude', 'Longitude','VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(all_venues1.shape)
all_venues1.head()

(26, 7)


,Community (English),Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Al Bada'a,25.224626,55.268483,Emergency الطواريء Iranian Hospital المستشفى ا...,25.230041,55.269398,Emergency Room
1,Al Baraha,25.281062,55.319466,Dubai Hospital (مستشفى دبي),25.284787,55.321645,Hospital
2,Al Buteen,25.263057,55.320584,canadian specialist hospital auditorium,25.263429,55.324375,Hospital
3,Al Dhagaya,25.272641,55.300241,Royal healthcare medical center dubai - Best H...,25.274504,55.299683,Acupuncturist
4,"Al Hamriya, Dubai",25.259930,55.303194,Medeor Hospital Dubai,25.257500,55.306492,Hospital


In [220]:
search_query2 = 'Food'
categoryId = '4d4b7105d754a06374d81259'
LIMIT = 1

In [221]:
def getNearbyVenues(names, lat1, long1, radius):
    venues_list=[]
    for name, lat, lng in zip(names, lat1, long1):

        # create the API request URL
        url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&locale={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query2, radius, LIMIT,  'en', categoryId)
        # make the GET request
        results = requests.get(url3).json()["response"]["venues"]
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    return(nearby_venues)

In [222]:
names=dfa['Community (English)']
latitudes=dfa['Latitude']
longitudes=dfa['Longitude']
all_venues2 = getNearbyVenues(names,latitudes, longitudes, radius )

In [223]:
# convert the venues list into a new DataFrame
# define the column names
all_venues2.columns = ['Community (English)','Latitude', 'Longitude','VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(all_venues2.shape)
all_venues2.head()

(29, 7)


,Community (English),Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Al Buteen,25.263057,55.320584,Al Diwan Lebanes Food,25.261937,55.319850,Middle Eastern Restaurant
1,Al Dhagaya,25.272641,55.300241,Kabul Darbar Restaurant (Afghan food),25.270802,55.303498,Middle Eastern Restaurant
2,"Al Hamriya, Dubai",25.259930,55.303194,Dalma Arabic Fast Food,25.262879,55.299978,Fast Food Restaurant
3,Al Hudaiba,25.242833,55.279212,Food Castle,25.243396,55.284155,Indian Restaurant
4,Al Karama,25.248230,55.305157,Food Ka Mood,25.248093,55.308846,North Indian Restaurant


In [224]:
#final_venues=[all_venues,all_venues1,all_venues2]
final_venues = pd.concat([all_venues,all_venues1,all_venues2])
final_venues = pd.concat([final_venues], ignore_index=True)
final_venues

,Community (English),Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.286029,55.328865,Hamriya Park,25.285710,55.333000,Park
1,Al Bada'a,25.224626,55.268483,Al Wasl Park (حديقة الوصل),25.222958,55.265674,Park
2,Al Baraha,25.281062,55.319466,Watchtower Park,25.276902,55.316955,Park
3,Al Buteen,25.263057,55.320584,Union Metro Park,25.263787,55.318791,Park
4,Al Garhoud,25.240737,55.352039,Garhoud Park,25.240850,55.349180,Park
...,...,...,...,...,...,...,...
78,Rigga Al Buteen,25.259523,55.319510,Al Diwan Lebanes Food,25.261937,55.319850,Middle Eastern Restaurant
79,Umm Suqeim First,25.153909,55.207500,Falla (فلة),25.157368,55.205578,Snack Place
80,Warsan First,25.163154,55.422077,华人通讯,25.163571,55.416758,Sri Lankan Restaurant
81,Za'abeel First,25.224255,55.286425,Food truck | DWTC,25.226054,55.283620,Food Truck


In [225]:
quality_infra_dubai = final_venues.copy()

In [226]:
quality_infra_dubai.to_csv('essentialinfra.csv')

In [227]:
quality_infra_dubai

,Community (English),Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abu Hail,25.286029,55.328865,Hamriya Park,25.285710,55.333000,Park
1,Al Bada'a,25.224626,55.268483,Al Wasl Park (حديقة الوصل),25.222958,55.265674,Park
2,Al Baraha,25.281062,55.319466,Watchtower Park,25.276902,55.316955,Park
3,Al Buteen,25.263057,55.320584,Union Metro Park,25.263787,55.318791,Park
4,Al Garhoud,25.240737,55.352039,Garhoud Park,25.240850,55.349180,Park
...,...,...,...,...,...,...,...
78,Rigga Al Buteen,25.259523,55.319510,Al Diwan Lebanes Food,25.261937,55.319850,Middle Eastern Restaurant
79,Umm Suqeim First,25.153909,55.207500,Falla (فلة),25.157368,55.205578,Snack Place
80,Warsan First,25.163154,55.422077,华人通讯,25.163571,55.416758,Sri Lankan Restaurant
81,Za'abeel First,25.224255,55.286425,Food truck | DWTC,25.226054,55.283620,Food Truck


<h2>CLUSTERING THE IMPORTED DATASET

In [228]:
quality_infra_dubai['VenueCategory'].unique()

array(['Park', 'Emergency Room', 'Hospital', 'Acupuncturist',
       "Dentist's Office", "Doctor's Office", 'Medical Center',
       'Middle Eastern Restaurant', 'Fast Food Restaurant',
       'Indian Restaurant', 'North Indian Restaurant', 'Burger Joint',
       'Food Court', 'Italian Restaurant', 'Food Truck', 'Tea Room',
       'Breakfast Spot', 'Comfort Food Restaurant',
       'Mediterranean Restaurant', 'Restaurant', 'Snack Place',
       'Sri Lankan Restaurant'], dtype=object)

In [229]:
# one hot encoding
quality_dubai_onehot = pd.get_dummies(quality_infra_dubai[['VenueCategory']], prefix="", prefix_sep="")

# add community to dataframe
quality_dubai_onehot['Community (English)'] = quality_infra_dubai['Community (English)'] 

# move column to the first column
fixed_columns = list(quality_dubai_onehot.columns[-1:]) + list(quality_dubai_onehot.columns[:-1])
quality_dubai_onehot = quality_dubai_onehot[fixed_columns]

print(quality_dubai_onehot.shape)
quality_dubai_onehot.head()

(83, 23)


,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,Emergency Room,Fast Food Restaurant,Food Court,...,Italian Restaurant,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room
0,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Al Bada'a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Al Baraha,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Al Buteen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Al Garhoud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [230]:
qualitydubai_grouped = quality_dubai_onehot.groupby(["Community (English)"]).sum().reset_index()
print(qualitydubai_grouped.shape)
qualitydubai_grouped.head()

(50, 23)


,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,Emergency Room,Fast Food Restaurant,Food Court,...,Italian Restaurant,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room
0,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Al Bada'a,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Al Baraha,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Al Buteen,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,Al Dhagaya,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [231]:
qualitydubai_grouped['Total infrastructure'] =  qualitydubai_grouped[qualitydubai_grouped.drop(['Community (English)'], axis=1).columns.values].sum(axis=1)

In [232]:
qualitydubai_groupedmax = qualitydubai_grouped[qualitydubai_grouped['Total infrastructure'] == qualitydubai_grouped['Total infrastructure'].max()]
print("Best place to stay within a city for vital infrastructure facilities :")
qualitydubai_groupedmax[['Community (English)', 'Total infrastructure']]

Best place to stay within a city for vital infrastructure facilities :


,Community (English),Total infrastructure
3,Al Buteen,3
13,Al Muraqqabat,3
14,Al Murar,3
29,Ayal Nasir,3
38,Mirdif,3
40,Naif,3


In [233]:
dubai_merged2 = qualitydubai_grouped.copy()
dubai_merged2 = dubai_merged2.join(dfa[["Community Number",'Latitude', 'Longitude', "Community (English)" ]].set_index("Community (English)"), on="Community (English)")

In [234]:
fixed_columns = list(dubai_merged2.columns[-3:]) + list(dubai_merged2.columns[:-3])
dubai_merged2 = dubai_merged2[fixed_columns]
print(dubai_merged2.shape)
dubai_merged2.head()

(50, 27)


,Community Number,Latitude,Longitude,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,...,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room,Total infrastructure
0,126,25.286029,55.3289,Abu Hail,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,333,25.224626,55.2685,Al Bada'a,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
2,122,25.281062,55.3195,Al Baraha,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
3,114,25.263057,55.3206,Al Buteen,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,3
4,113,25.272641,55.3002,Al Dhagaya,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2


In [235]:
# set number of clusters
kclusters = 3

dubai_2_grouped_clustering = dubai_merged2[["Total infrastructure"]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dubai_2_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 1, 2, 0, 2, 2, 0, 2], dtype=int32)

In [236]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each community
dubai_mergedfinal = dubai_merged2.copy()
# add clustering labels
dubai_mergedfinal["Cluster Labels"] = kmeans.labels_
print(dubai_mergedfinal.shape)
dubai_mergedfinal.head() # check the last column

(50, 28)


,Community Number,Latitude,Longitude,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,...,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room,Total infrastructure,Cluster Labels
0,126,25.286029,55.3289,Abu Hail,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,333,25.224626,55.2685,Al Bada'a,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
2,122,25.281062,55.3195,Al Baraha,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
3,114,25.263057,55.3206,Al Buteen,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,3,1
4,113,25.272641,55.3002,Al Dhagaya,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,2


<h2>11. Exploratory Visualization 2

In [237]:
# Set manually to get proper fit in the map
address = 'Dubai'
latitude =  25.276987
longitude =  25.276987
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Dubai are 25.276987, 25.276987.


In [238]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'orange',    'darkgreen',    'darkblue',    'black']
# add markers to map
markers_colors = []
for lat, lng, label1,common, cluster in zip(dubai_mergedfinal['Latitude'], dubai_mergedfinal['Longitude'], dubai_mergedfinal['Community (English)'],dubai_mergedfinal['Total infrastructure'] , dubai_mergedfinal['Cluster Labels']):
    labelnew =  'Community (English) : {} , Total infrastructure : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

<h2>12. Examine Clusters</h2>

<h4>Cluster 0
    

In [239]:
dubai_mergedfinal.loc[dubai_mergedfinal['Cluster Labels'] == 0]

,Community Number,Latitude,Longitude,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,...,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room,Total infrastructure,Cluster Labels
0,126,25.286029,55.3289,Abu Hail,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5,214,25.240737,55.352,Al Garhoud,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
8,323,25.233594,55.2922,Al Jafiliya,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
10,134,25.314692,55.3416,Al Mamzar,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
11,317,25.249509,55.2955,Al Mankhool,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
15,252,25.214260,55.4507,Al Mushrif,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
18,316,25.254206,55.288,Al Raffa,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
19,112,25.268643,55.2955,Al Ras,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
20,216,25.222025,55.391,Al Rashidiya,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
21,119,25.267208,55.3102,Al Rigga,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


<h4>Cluster 1

In [240]:
dubai_mergedfinal.loc[dubai_mergedfinal['Cluster Labels'] == 1]

,Community Number,Latitude,Longitude,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,...,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room,Total infrastructure,Cluster Labels
3,114,25.263057,55.3206,Al Buteen,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,3,1
13,124,25.265539,55.3225,Al Muraqqabat,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,3,1
14,117,25.276739,55.3101,Al Murar,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,3,1
29,116,25.274348,55.3041,Ayal Nasir,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,3,1
38,251,25.221171,55.4224,Mirdif,0,0,0,0,0,1,...,0,0,0,1,1,0,0,0,3,1
40,118,25.271865,55.3102,Naif,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,3,1


<h4>Cluster 2

In [241]:
dubai_mergedfinal.loc[dubai_mergedfinal['Cluster Labels'] == 2]

,Community Number,Latitude,Longitude,Community (English),Acupuncturist,Breakfast Spot,Burger Joint,Comfort Food Restaurant,Dentist's Office,Doctor's Office,...,Mediterranean Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Park,Restaurant,Snack Place,Sri Lankan Restaurant,Tea Room,Total infrastructure,Cluster Labels
1,333,25.224626,55.2685,Al Bada'a,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
2,122,25.281062,55.3195,Al Baraha,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
4,113,25.272641,55.3002,Al Dhagaya,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,2
6,313,25.259930,55.3032,"Al Hamriya, Dubai",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
7,322,25.242833,55.2792,Al Hudaiba,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
9,318,25.248230,55.3052,Al Karama,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,2,2
12,262,25.245084,55.4503,Al Mizhar First,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,2,2
16,123,25.274154,55.3226,Al Muteena,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
17,231,25.273208,55.3695,Al Nahda First,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,2
22,115,25.269226,55.3025,Al Sabkha,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,2
